In [1]:
import unittest
from test.aqua import QiskitAquaTestCase
import numpy as np
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit import(QuantumCircuit, execute, Aer)
from qiskit.aqua.algorithms import VQC
from qiskit import *
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.aqua import QuantumInstance
from qiskit.circuit import ParameterVector
from qiskit import Aer, execute
import matplotlib.pyplot as plt
from math import pi
import numpy as np
from torch import nn
import torch
import torch.nn.functional as F
from qiskit import BasicAer
from qiskit.aqua.algorithms import QuantumFisherInf
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter, Gate, ControlledGate, Qubit
from qiskit.extensions.standard import RXGate, RYGate, RZGate, CRZGate
import numpy as np
from qiskit.aqua import QuantumInstance, aqua_globals

In [ ]:
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000
np.random.seed(42)

# data
inputsize=2
depth=1
datasize=1
blocks=1
x = np.random.normal(0, 1, size=inputsize)

params = ParameterVector(name='params', length=12)
q = QuantumRegister(inputsize, name='q')
featuremap = SecondOrderExpansion(feature_dimension=inputsize, depth=depth)
circuit = featuremap.construct_circuit(x, qr=q)  # Use a specific register to make ensure functionality of compose see below
circuit.barrier()
model = QuantumCircuit(inputsize)

model.rx(params[0],0)
model.rx(params[1],1)
model.ry(params[2],0)
model.ry(params[3],1)
model.rz(params[4],0)
model.rz(params[5],1)

for i in range(inputsize):
    if i + 1 > inputsize - 1:
        break
    model.cz(i, i + 1)
model.cz(0, inputsize - 1)

model.rx(params[6],0)
model.rx(params[7],1)
model.ry(params[8],0)
model.ry(params[9],1)
model.rz(params[10],0)
model.rz(params[11],1)


# Check this part --> use compose
        
print(circuit.data)
print('--------------------------------')
print(circuit)
circuit.compose(model, qubits=q, inplace=True)
print(model.data)
parameterized_gates = []
for param, elements in circuit._parameter_table.items():
    for element in elements:
        parameterized_gates.append(element[0])
print('--------------------------------')
print(parameterized_gates)
print('--------------------------------')
print(circuit.data)
print('--------------------------------')
print(circuit)

#---------------

def create_rand_params(inputsize, blocks, seed):
    np.random.seed(seed)
    param_length = inputsize * 3 + inputsize * blocks * 3
    return np.random.uniform(-2*pi, 2*pi, param_length)


In [6]:
circuit.decompose().draw()

<pre style="word-wrap: normal;white-space: pre;background: #fff0;line-height: 1.1;font-family: &quot;Courier New&quot;,Courier,monospace">     ┌───────────────┐┌─────────────┐                         ░ »
q_0: ┤ U3(pi/2,0,pi) ├┤ U1(0.99343) ├───■─────────────────■───░─»
     ├───────────────┤├─────────────┴┐┌─┴─┐┌───────────┐┌─┴─┐ ░ »
q_1: ┤ U3(pi/2,0,pi) ├┤ U1(-0.27653) ├┤ X ├┤ U1(17.35) ├┤ X ├─░─»
     └───────────────┘└──────────────┘└───┘└───────────┘└───┘ ░ »
«     ┌───────────────┐┌───────────────┐┌───────────────┐      »
«q_0: ┤ Rx(params[0]) ├┤ Ry(params[2]) ├┤ Rz(params[4]) ├─■──■─»
«     ├───────────────┤├───────────────┤├───────────────┤ │  │ »
«q_1: ┤ Rx(params[1]) ├┤ Ry(params[3]) ├┤ Rz(params[5]) ├─■──■─»
«     └───────────────┘└───────────────┘└───────────────┘      »
«     ┌───────────────┐┌───────────────┐┌────────────────┐
«q_0: ┤ Rx(params[6]) ├┤ Ry(params[8]) ├┤ Rz(params[10]) ├
«     ├───────────────┤├───────────────┤├────────────────┤
«q_1: ┤ Rx(params[7]) ├┤ Ry(params[9]) ├┤ Rz(params[11]) ├
«     └───────────────┘└───────────────┘└────────────────┘</pre>

In [4]:
params

ParameterVector(name=params, length=12)

In [7]:
values = create_rand_params(inputsize, blocks, seed=42)

qi = QuantumInstance(backend=BasicAer.get_backend('statevector_simulator'), seed_simulator=2,seed_transpiler=2)

qfi = QuantumFisherInf(circuit=circuit.decompose(), quantum_instance=qi)
# qfi.construct_circuits(params)
qfi.compute_qfi(params, values)

I may have done a hacky thing with the construct_evaluation_circuit method of the weighted pauli operator. I.e. construct circuit did not work with lists of quantum registers but only full registers. Wait for refactoring of operator.
Testing  [<qiskit.extensions.standard.rx.RXGate object at 0x137282588>, <qiskit.extensions.standard.ry.RYGate object at 0x1372858d0>, <qiskit.extensions.standard.rz.RZGate object at 0x137285cc0>, <qiskit.extensions.standard.rx.RXGate object at 0x137285438>, <qiskit.extensions.standard.ry.RYGate object at 0x137285828>, <qiskit.extensions.standard.rz.RZGate object at 0x137285a58>, <qiskit.extensions.standard.rx.RXGate object at 0x137285a20>, <qiskit.extensions.standard.ry.RYGate object at 0x137285128>, <qiskit.extensions.standard.rz.RZGate object at 0x137285400>, <qiskit.extensions.standard.rx.RXGate object at 0x1372857b8>, <qiskit.extensions.standard.ry.RYGate object at 0x1372859b0>, <qiskit.extensions.standard.rz.RZGate object at 0x1372855f8>]


/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  return self._paulis[0][1].numberofqubits
/Users/ouf/Documents/GitHub/qiskit-aqua-varqte/qiskit/aqua/operators/weighted_pauli_operator.py:118: DeprecationWarning: The Pauli.numberofqubits method is deprecated as of 0.13.0, and will be removed no earlier than 3 months after that release date. You should use the Pauli.num_qubits method instead.
  ret

array([[ 9.98505481e-01+0.00000000e+00j,  2.23579118e-03+6.93889390e-17j,
         3.68147504e-01+5.55111512e-17j,  5.22450124e-01+2.77555756e-17j,
         8.36640726e-01+2.77555756e-17j, -4.56683641e-01+1.11022302e-16j,
         7.57061790e-01+5.55111512e-17j,  2.79035198e-01+4.16333634e-17j,
        -9.43178763e-01+1.11022302e-16j, -8.84663942e-01+1.11022302e-16j,
         4.16283513e-01+4.16333634e-17j, -8.57781165e-01+1.11022302e-16j],
       [ 2.23579118e-03+6.93889390e-17j,  9.96655271e-01+0.00000000e+00j,
         1.54101080e-03+0.00000000e+00j, -7.20485593e-01+7.63278329e-17j,
         5.28036756e-01+1.38777878e-17j,  5.58139046e-01+2.77555756e-17j,
         5.80018572e-01+1.38777878e-17j, -7.01353144e-01+5.55111512e-17j,
        -8.40265722e-02+5.55111512e-17j, -2.47941488e-01+4.16333634e-17j,
        -7.83471869e-01+1.11022302e-16j, -1.84179684e-01+7.63278329e-17j],
       [ 3.68147504e-01+5.55111512e-17j,  1.54101080e-03+0.00000000e+00j,
         9.99290013e-01+0.00000000e+